## AAI 627: Data Acquisition, Modeling and Analysis

## Fall 2022: Project Assignment

### Driver Drowsiness Alert System using Deep Learning
### Part 2: Model Deployment and Webcam Script

In [1]:
import cv2 as cv
import numpy as np
import face_recognition
from tensorflow import keras
import config as cf
from os import path
from playsound import playsound

### disable prediction printing of prediction bar as stdout

keras.utils.disable_interactive_logging()


trained_model = keras.models.load_model(path.join(cf.TRAINED_MODEL_PATH,"Model_relu.h5"))

def eye_extracted(frame):
    facial_feature_list = face_recognition.face_landmarks(frame)

    try:
        eye_feature = facial_feature_list[0]['left_eye']
    except Exception as e:
        try:
            eye_feature = facial_feature_list[0]['right_eye']
        except:
            return

    x_max = max([coordinate[0] for coordinate in eye_feature])
    x_min = min([coordinate[0] for coordinate in eye_feature])
    y_max = max([coordinate[1] for coordinate in eye_feature])
    y_min = min([coordinate[1] for coordinate in eye_feature])

    x_range = x_max - x_min
    y_range = y_max - y_min

    # adding 50% padding to axis with larger ranger

    if x_range > y_range:
        right = round(.5*x_range) + x_max
        left = x_min - round(.5*x_range)
        bottom = round((((right-left) - y_range))/2) + y_max
        top = y_min - round((((right-left) - y_range))/2)
    else:
        bottom = round(.5*y_range) + y_max
        top = y_min - round(.5*y_range)
        right = round((((bottom-top) - x_range))/2) + x_max
        left = x_min - round((((bottom-top) - x_range))/2)

    cropped = frame[top:(bottom + 1), left:(right + 1)]
    cropped = cv.resize(cropped, (80, 80))
    return cropped.reshape(-1, 80, 80, 3)


In [5]:
def main_function():
    vid_frame = cv.VideoCapture(0)
    vid_frame.set(cv.CAP_PROP_FPS, 20)
    vid_frame.set(cv.CAP_PROP_FRAME_WIDTH, 600)
    vid_frame.set(cv.CAP_PROP_FRAME_HEIGHT, 300)
    

    w = vid_frame.get(cv.CAP_PROP_FRAME_WIDTH)
    h = vid_frame.get(cv.CAP_PROP_FRAME_HEIGHT)
    print("Video Frames:"+str(vid_frame.get(cv.CAP_PROP_FPS)))
    print("Video codec :"+str(vid_frame.get(cv.CAP_PROP_FOURCC)))

    counter = 0

    if not vid_frame.isOpened():
        print("Error opening Webcam")

    while True:

        
        ret, frame = vid_frame.read()
        frame = cv.flip(frame,1)
        img_predict = eye_extracted(frame)
        try:
            img_predict=img_predict/255.0
        except Exception as e:
            status = 'Eye ROI not Detected'
            
            cv.rectangle(frame, (round(w/2) - 110, 10),
                         (round(w/2) + 110, 90), (38, 38, 38), -1)
            cv.putText(frame, status, (round(w/2)-80, 70),
                       cv.FONT_HERSHEY_DUPLEX, 2, (0, 255, 0), 2, cv.LINE_4)
            continue
        prediction_score =trained_model.predict(img_predict)
        
        if prediction_score<0.5:
            status = 'Open'
            counter = 0

            cv.rectangle(frame, (round(w/2) - 110,10), (round(w/2) + 110, 90), (38,38,38), -1)
            cv.putText(frame, status, (round(w/2)-80, 70),
                       cv.FONT_HERSHEY_DUPLEX, 2, (0, 255, 0), 2, cv.LINE_4)
          
        else:
            status = 'Close'
            counter+=1
            cv.rectangle(frame, (round(w/2) - 110, 10),
                         (round(w/2) + 110, 100), (38, 38, 38), -1)
            cv.putText(frame, status, (round(w/2)-80, 70),
                       cv.FONT_HERSHEY_DUPLEX, 2, (0, 0, 255), 2, cv.LINE_4)

            if counter>3:
                playsound(os.path.join(cf.WORKING_DIR,'airhorn.mp3'))
                counter=1
        cv.imshow('Drowsiness Detection', frame)

        k = cv.waitKey(1)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    vid_frame.release()
    cv.destroyAllWindows()


main_function()

Video Frames:30.0
Video codec :22.0


ERROR:playsound:
    Error 305 for command:
        close "C:\Users\PRIYAN~1\AppData\Local\Temp\PSjjglmjay.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.
ERROR:playsound:
    Error 305 for command:
        close "C:\Users\PRIYAN~1\AppData\Local\Temp\PS9e9fxz54.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.
ERROR:playsound:
    Error 305 for command:
        close "C:\Users\PRIYAN~1\AppData\Local\Temp\PSx6_q13od.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.
ERROR:playsound:
    Error 305 for command:
        close "C:\Users\PRIYAN~1\AppData\Local\Temp\PS3sqksih5.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.
ERROR:playsound:
    Error 305 for command:
        close "C:\Users\PRIYAN~1\AppData\Local\Temp\PSx6ohcfby.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.
ERROR:playsound:
    Error 305 for comma

In [3]:
import os
from playsound import playsound

